# **Modelo de Naive Bayes**
Este modelo es ampliamente utilizado para la clasificación de textos como para el análisis de sentimientos, incluso aunque estos sean multiclase. Las posibilidades que tenemos con este modelo de acuerdo a nuestro problema son las siguientes:

| Modelo                    | Supuestos                                         | Ejemplos de Uso                                           | Importación en Python                            |
|---------------------------|---------------------------------------------------|-----------------------------------------------------------|--------------------------------------------------|
| Gaussian Naive Bayes      | Características siguen distribución gaussiana     | Clasificación de correos electrónicos, diagnóstico médico  | `from sklearn.naive_bayes import GaussianNB`      |
| Multinomial Naive Bayes   | Características discretas, como frecuencias de palabras | Análisis de sentimientos, clasificación de documentos    | `from sklearn.naive_bayes import MultinomialNB`   |
| Bernoulli Naive Bayes     | Datos binarios (0 o 1)                             | Clasificación de documentos binarios                      | `from sklearn.naive_bayes import BernoulliNB`     |
| Complement Naive Bayes    | Similar a MultinomialNB, pero para clases desequilibradas | Clasificación de texto con clases desequilibradas      | `from sklearn.naive_bayes import ComplementNB`    |
| Categorical Naive Bayes   | Datos categóricos                                  | Clasificación de encuestas                                | `from sklearn.naive_bayes import CategoricalNB`  |

Veamos algunos ejemplos


## **Ejercicio1**
Este primer proyecto consiste en predecir si un estudiante aprobará o no el curso dadas las horas de estudio y su asistencia a clase, por lo tanto estamos ante un problema de clasificación binaria. Usemos el modelo de Naive Bayes para solucionarlo

In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# Generar aleatoriamente 50 registros de datos
import random

data = []
for _ in range(50):
    horas_estudio = random.uniform(1, 10)
    asistencia_clases = random.uniform(50, 100)
    aprobado = 'Sí' if random.choice([True, False]) else 'No'
    data.append([horas_estudio, asistencia_clases, aprobado])

# Convertir los datos en un DataFrame de pandas
columnas = ['Horas de Estudio', 'Asistencia a Clases', 'Aprobado']
df = pd.DataFrame(data, columns=columnas)

In [21]:
df.head(2)

,Horas de Estudio,Asistencia a Clases,Aprobado
0,4.311461,61.898953,Sí
1,1.559659,66.888815,Sí


Para el clasificador Naive Bayes, en particular, no es estrictamente necesario realizar escalado de características. Los clasificadores Naive Bayes, incluido el Naive Bayes Gaussiano que estamos utilizando, asumen que las características son independientes entre sí dado el valor de la clase, y no se ven afectados por la escala de las características.

El escalado de características, como la normalización o la estandarización, suele ser importante para algoritmos basados en distancias o gradientes, como la regresión logística o las máquinas de soporte vectorial. Sin embargo, Naive Bayes es menos sensible a la escala de las características debido a su suposición "ingenua" de independencia condicional.

In [22]:
# Dividir los datos en características (X) y etiquetas (y)
X = df[['Horas de Estudio', 'Asistencia a Clases']]
y = df['Aprobado']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_entrenamiento, X_prueba, y_entrenamiento, y_prueba = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el clasificador Naive Bayes
clf = GaussianNB()
clf.fit(X_entrenamiento, y_entrenamiento)

# Realizar predicciones en el conjunto de prueba
predicciones = clf.predict(X_prueba)

# Calcular la precisión
precision = accuracy_score(y_prueba, predicciones)
print(f"Precisión del modelo: {precision * 100:.2f}%")


Precisión del modelo: 70.00%


In [23]:
X_prueba["real"] = y_prueba
X_prueba["predicciones"] = predicciones
X_prueba

,Horas de Estudio,Asistencia a Clases,real,predicciones
13,3.761208,59.995248,No,Sí
39,1.962585,53.707848,Sí,Sí
30,6.152596,67.130252,Sí,Sí
45,8.166968,51.697111,Sí,Sí
17,4.856146,83.761839,Sí,Sí
48,7.081241,86.333303,No,No
26,5.643757,86.389536,Sí,No
25,1.299697,79.014257,No,Sí
32,9.088833,95.558083,No,No
19,2.768639,86.156289,Sí,Sí


## **Ejercicio 2**
Ahora utilizaremos el modelo de Naive Bayes para analizar sentimientos.veamos priemro que todo cómo funciona este modelo. Para implementar el análisis de sentimientos usando Naive Bayes necesitamos hacer uso del método `CountVectorizer` de `sklearn`, este método convierte una frase en un vector, el `corpus` del vectorizador debe ser el conjunto de frases que se usarán en nuestro modelo, posteriormente cada frase se convertirá en un vector de tamaño n, donde n es la cantidad de palabras únicas presentes en el `corpus` y cada elemento del vector es un número que indica cuantas veces aparece cada palabra en la frase. veamos cómo funciona entonces el `CountVectorizer`

In [24]:
from sklearn.feature_extraction.text import CountVectorizer

# Ejemplo de datos
corpus = ["Este es un ejemplo de un codigo.", "Otro ejemplo.", "Y otro más."]

# Crear el vectorizador
vectorizador = CountVectorizer()

# Ajustar y transformar el corpus
X = vectorizador.fit_transform(corpus)

# Obtener el vocabulario
vocabulario = vectorizador.get_feature_names_out()

# Mostrar la matriz de recuentos
matriz_recuentos = X.toarray()
print("Vocabulario:", vocabulario)
print("Matriz de recuentos:")
print(matriz_recuentos)

Vocabulario: ['codigo' 'de' 'ejemplo' 'es' 'este' 'más' 'otro' 'un']
Matriz de recuentos:
[[1 1 1 1 1 0 0 2]
 [0 0 1 0 0 0 1 0]
 [0 0 0 0 0 1 1 0]]


Como podemos observar el corpus de nuestro modelo es `corpus: codigo, de, ejemplo, es, este, más, otro, un` en este orden se construirá cada vector, notemos que en la primera frase la palabra `un` aparece 2 veces, es por esta razón que el último elemento del vector correspondiente a la primera frase tiene como valor numérico 2. la parabra `más` y `otro` no aparecen, por lo tanto sus veloces son cero para las posiciones correspondientes del vector y las demás aparecen solo una vez. Estos vectores son los usados para entrenar el modelo de Machine Learning, por lo tanto cada palabra única se convierte en una feature para nuestro modelo.

# **Ejercicio 3**
Ahora entrenemos un modelo de Naive Bayes y hagamos análisis de sentimientos

In [25]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Datos inventados
data = [
    {"comentario": "La aplicación es muy fácil de usar y eficiente.", "sentimiento": "positivo"},
    {"comentario": "No me gusta la interfaz, es complicada de entender.", "sentimiento": "negativo"},
    {"comentario": "Buena aplicación, pero a veces se cuelga.", "sentimiento": "mixto"},
    {"comentario": "Hasta ahora, todo ha sido perfecto.", "sentimiento": "positivo"},
    {"comentario": "No encuentro útiles las funciones ofrecidas.", "sentimiento": "negativo"},
    {"comentario": "Podría mejorar en términos de personalización.", "sentimiento": "neutro"},
    {"comentario": "Funciona bien en general, pero tiene algunos problemas.", "sentimiento": "mixto"},
    {"comentario": "¡Increíble! La mejor aplicación que he probado.", "sentimiento": "positivo"},
    {"comentario": "Servicio al cliente terrible, no resolvieron mi problema.", "sentimiento": "negativo"},
    {"comentario": "No está mal, pero le falta innovación.", "sentimiento": "neutro"},
    {"comentario": "Muy satisfecho con la rapidez del servicio.", "sentimiento": "positivo"},
    {"comentario": "No puedo entender cómo usar esta aplicación. Es frustrante.", "sentimiento": "negativo"},
    {"comentario": "Es decente. Algunas características son buenas, otras no tanto.", "sentimiento": "neutro"},
    {"comentario": "Excelente servicio al cliente. Resolvieron mi problema rápidamente.", "sentimiento": "positivo"},
    {"comentario": "No hay suficientes opciones de personalización. Decepcionado.", "sentimiento": "negativo"},
    {"comentario": "No encuentro ninguna falla importante en la aplicación.", "sentimiento": "positivo"},
    {"comentario": "Me confunden las constantes actualizaciones.", "sentimiento": "negativo"},
    {"comentario": "La aplicación es bastante básica, pero funciona bien.", "sentimiento": "neutro"},
    {"comentario": "¡Increíble! No puedo vivir sin esta aplicación.", "sentimiento": "positivo"},
    {"comentario": "Demasiadas notificaciones, resulta molesto.", "sentimiento": "negativo"},
    {"comentario": "No está mal, pero a veces se congela.", "sentimiento": "mixto"},
    {"comentario": "La aplicación ha mejorado mucho desde la última actualización.", "sentimiento": "positivo"},
    {"comentario": "No cumplió con mis expectativas. No la recomendaría.", "sentimiento": "negativo"},
    {"comentario": "Es decente, pero podría ser mejor.", "sentimiento": "neutro"},
    {"comentario": "Funciona bien, pero la interfaz es anticuada.", "sentimiento": "mixto"},
    {"comentario": "¡Fantástico! Todo lo que necesito en una aplicación.", "sentimiento": "positivo"},
    {"comentario": "No entiendo por qué esta aplicación tiene tantas reseñas positivas.", "sentimiento": "negativo"},
    {"comentario": "Aceptable, pero no es excepcional.", "sentimiento": "neutro"},
    {"comentario": "La aplicación se bloquea con frecuencia, es frustrante.", "sentimiento": "mixto"},
    {"comentario": "Me encanta la última actualización. ¡Gran trabajo!", "sentimiento": "positivo"},
    {"comentario": "Pésima experiencia de usuario. No recomendaría.", "sentimiento": "negativo"},
    {"comentario": "No está mal, pero hay espacio para mejoras.", "sentimiento": "neutro"},
    {"comentario": "Funciona correctamente, pero la velocidad podría mejorar.", "sentimiento": "mixto"},
    {"comentario": "Increíble servicio al cliente. Siempre están disponibles.", "sentimiento": "positivo"},
    {"comentario": "No vale la pena. Hay mejores opciones disponibles.", "sentimiento": "negativo"},
    {"comentario": "Es aceptable, pero no destaca en nada en particular.", "sentimiento": "neutro"},
    {"comentario": "Algunas características son buenas, otras son confusas.", "sentimiento": "mixto"},
    {"comentario": "La aplicación ha superado mis expectativas.", "sentimiento": "positivo"},
    {"comentario": "Terrible atención al cliente. No resolvieron mi problema.", "sentimiento": "negativo"},
    {"comentario": "Funciona bien, pero me gustaría más personalización.", "sentimiento": "neutro"},
    {"comentario": "La aplicación tiene un buen diseño, pero algunas funciones son difíciles de encontrar.", "sentimiento": "mixto"},
    {"comentario": "¡Me encanta! La mejor aplicación que he probado.", "sentimiento": "positivo"},
    {"comentario": "No entiendo por qué tantas personas la consideran útil.", "sentimiento": "negativo"},
    {"comentario": "Cumple con su propósito, pero hay opciones mejores.", "sentimiento": "neutro"},
    {"comentario": "A veces funciona bien, a veces no tanto.", "sentimiento": "mixto"},
    {"comentario": "La aplicación ha mejorado mi productividad significativamente.", "sentimiento": "positivo"},
    {"comentario": "Definitivamente no recomendaría esta aplicación.", "sentimiento": "negativo"},
    {"comentario": "Es decente, pero necesita más funciones.", "sentimiento": "neutro"},
    {"comentario": "Algunas características son útiles, otras no tanto.", "sentimiento": "mixto"},
]

In [26]:
# Convertir los datos en un DataFrame de pandas
df = pd.DataFrame(data)

In [27]:
# Dividir los datos en características (X) y etiquetas (y)
X = df['comentario']
y = df['sentimiento']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_entrenamiento, X_prueba, y_entrenamiento, y_prueba = train_test_split(X, y, test_size=0.2, random_state=42)

In [28]:
# Vectorizar los comentarios
vectorizador = CountVectorizer()
X_entrenamiento_vect = vectorizador.fit_transform(X_entrenamiento)
X_prueba_vect = vectorizador.transform(X_prueba)

In [29]:
# Obtener el vocabulario directamente desde el vectorizador
corpus = vectorizador.get_feature_names_out()

# Mostrar el vocabulario
print("Vocabulario:", corpus)

Vocabulario: ['aceptable' 'actualizaciones' 'actualización' 'ahora' 'al' 'algunas'
 'algunos' 'anticuada' 'aplicación' 'atención' 'bien' 'bloquea' 'buen'
 'buena' 'buenas' 'características' 'cliente' 'complicada' 'con'
 'confunden' 'confusas' 'congela' 'consideran' 'constantes'
 'correctamente' 'cuelga' 'cumple' 'cumplió' 'cómo' 'de' 'decente'
 'decepcionado' 'definitivamente' 'del' 'desde' 'destaca' 'difíciles'
 'diseño' 'disponibles' 'eficiente' 'en' 'encanta' 'encontrar' 'encuentro'
 'entender' 'entiendo' 'es' 'esta' 'está' 'están' 'expectativas'
 'experiencia' 'falla' 'falta' 'frecuencia' 'frustrante' 'funciona'
 'funciones' 'fácil' 'general' 'gran' 'gusta' 'gustaría' 'ha' 'hasta'
 'hay' 'he' 'importante' 'increíble' 'innovación' 'interfaz' 'la' 'las'
 'le' 'mal' 'me' 'mejor' 'mejorado' 'mejorar' 'mejores' 'mi' 'mis' 'mucho'
 'muy' 'más' 'nada' 'ninguna' 'no' 'ofrecidas' 'opciones' 'otras'
 'particular' 'pena' 'perfecto' 'pero' 'personalización' 'personas'
 'podría' 'por' 'probado'

In [30]:
clf = MultinomialNB()

In [31]:
# Ejemplo de entrenamiento
clf.fit(X_entrenamiento_vect, y_entrenamiento)


MultinomialNB()

In [32]:
predicciones = clf.predict(X_prueba_vect)
probabilidades = clf.predict_proba(X_prueba_vect)

clf.classes_,probabilidades

(array(['mixto', 'negativo', 'neutro', 'positivo'], dtype='<U8'),
 array([[0.00245561, 0.96708415, 0.00328756, 0.02717267],
        [0.02943865, 0.05831947, 0.00180008, 0.9104418 ],
        [0.18640264, 0.02344592, 0.78157283, 0.00857861],
        [0.9202686 , 0.01496067, 0.060842  , 0.00392873],
        [0.94161077, 0.00681274, 0.02401187, 0.02756462],
        [0.19634013, 0.11627637, 0.66844275, 0.01894076],
        [0.02632243, 0.93952244, 0.00469264, 0.0294625 ],
        [0.07106104, 0.02549301, 0.03969137, 0.86375458],
        [0.2359275 , 0.09644442, 0.65948469, 0.00814338],
        [0.23076923, 0.30769231, 0.17948718, 0.28205128]]))

In [33]:
probaT = probabilidades.T
probaT

array([[0.00245561, 0.02943865, 0.18640264, 0.9202686 , 0.94161077,
        0.19634013, 0.02632243, 0.07106104, 0.2359275 , 0.23076923],
       [0.96708415, 0.05831947, 0.02344592, 0.01496067, 0.00681274,
        0.11627637, 0.93952244, 0.02549301, 0.09644442, 0.30769231],
       [0.00328756, 0.00180008, 0.78157283, 0.060842  , 0.02401187,
        0.66844275, 0.00469264, 0.03969137, 0.65948469, 0.17948718],
       [0.02717267, 0.9104418 , 0.00857861, 0.00392873, 0.02756462,
        0.01894076, 0.0294625 , 0.86375458, 0.00814338, 0.28205128]])

In [34]:
df_prueba = pd.DataFrame({"frase":X_prueba, "reales": y_prueba, "predicciones": predicciones,
                          "proba mixto":probaT[0], "proba negativo":probaT[1], "proba neutro":probaT[2],
                          "proba positivo":probaT[3]})
df_prueba

,frase,reales,predicciones,proba mixto,proba negativo,proba neutro,proba positivo
13,Excelente servicio al cliente. Resolvieron mi ...,positivo,negativo,0.002456,0.967084,0.003288,0.027173
45,La aplicación ha mejorado mi productividad sig...,positivo,positivo,0.029439,0.058319,0.001800,0.910442
47,"Es decente, pero necesita más funciones.",neutro,neutro,0.186403,0.023446,0.781573,0.008579
44,"A veces funciona bien, a veces no tanto.",mixto,mixto,0.920269,0.014961,0.060842,0.003929
17,"La aplicación es bastante básica, pero funcion...",neutro,mixto,0.941611,0.006813,0.024012,0.027565
27,"Aceptable, pero no es excepcional.",neutro,neutro,0.196340,0.116276,0.668443,0.018941
26,No entiendo por qué esta aplicación tiene tant...,negativo,negativo,0.026322,0.939522,0.004693,0.029462
25,¡Fantástico! Todo lo que necesito en una aplic...,positivo,positivo,0.071061,0.025493,0.039691,0.863755
31,"No está mal, pero hay espacio para mejoras.",neutro,neutro,0.235928,0.096444,0.659485,0.008143
19,"Demasiadas notificaciones, resulta molesto.",negativo,negativo,0.230769,0.307692,0.179487,0.282051


In [35]:
# Nuevo comentario que deseas predecir
nuevo_comentario = ["Esta aplicación es realmente buena, me gusta mucho."]

# Vectorizar el nuevo comentario utilizando el mismo vectorizador
nuevo_comentario_vect = vectorizador.transform(nuevo_comentario)

# Hacer la predicción
prediccion_nuevo_comentario = clf.predict(nuevo_comentario_vect)
probabilidades_nuevo_comentario = clf.predict_proba(nuevo_comentario_vect)

# Mostrar la predicción y las probabilidades
print("Sentimiento Predicho para el Nuevo Comentario:", prediccion_nuevo_comentario[0])
print("Probabilidades para el Nuevo Comentario:")
print(pd.DataFrame(probabilidades_nuevo_comentario, columns=clf.classes_))

Sentimiento Predicho para el Nuevo Comentario: positivo
Probabilidades para el Nuevo Comentario:
      mixto  negativo    neutro  positivo
0  0.073739  0.371214  0.036025  0.519022


### **Tener en cuenta para hacer nuevas predicciones**

**Vectorizador** (CountVectorizer):

- Ignora palabras nuevas en los datos de predicción que no estaban en el corpus de entrenamiento.
- Por defecto, solo se ajusta al vocabulario presente en los datos de entrenamiento.

**Clasificador** (MultinomialNB):

- Asume que las características en los datos de prueba siguen la misma distribución que en los datos de entrenamiento.
- Si una palabra nueva aparece durante la predicción y no estaba en el entrenamiento, el clasificador la ignorará.

**Manejo de Palabras Nuevas:**

- Para manejar palabras nuevas de manera efectiva, considera reentrenar el modelo con nuevos datos que incluyan esas palabras.
- También puedes explorar técnicas más avanzadas de procesamiento de texto que puedan manejar la variabilidad léxica.